In [1]:
import speech_recognition as sr
import pyttsx3
import nltk
import json
import tkinter as tk
from tkinter import scrolledtext
import random 
import pickle 
import numpy as np 
from nltk.stem import WordNetLemmatizer  
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Activation , Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model


In [ ]:

# Assurez-vous d'avoir les ressources nécessaires de nltk
nltk.download('punkt')
nltk.download('wordnet')

# Initialiser le lemmatiseur
lemmatizer = nltk.WordNetLemmatizer()

# Charger les intents
with open('intents2.json', 'r', encoding='utf-8') as file:
    intents = json.load(file)

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

# Traitement des données
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatization et suppression des signes de ponctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

# Sauvegarde des mots et des classes
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# Préparer les données d'entraînement
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

# Vérifiez les tailles des vecteurs
print("Shape of bags and output rows:")
for item in training:
    print(len(item[0]), len(item[1]))

# Convertir en tableaux NumPy
train_x = np.array([x for x, _ in training])
train_y = np.array([y for _, y in training])

# Créer le modèle
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Créer un optimiseur SGD
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

# Compiler le modèle
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
hist = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

# Sauvegarder le modèle
model.save('chatbot_model.h5', hist)
print('Done')

# Initialiser le moteur de synthèse vocale
speaker = pyttsx3.init()

# Initialiser le recognizer pour la reconnaissance vocale
recognizer = sr.Recognizer()

# Liste pour stocker les tâches à faire
todo_list = ["fakher ", "bhim"]

# Charger les intents et le modèle
with open('intents2.json', 'r', encoding='utf-8') as file:
    intents = json.load(file)
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.h5')

# Prétraitement des données
def preprocess_intents(intents):
    words = []
    classes = []
    documents = []
    ignore_letters = ['?', '!', '.', ',']

    for intent in intents['intents']:
        for pattern in intent['patterns']:
            word_list = nltk.word_tokenize(pattern)
            words.extend(word_list)
            documents.append((word_list, intent['tag']))
            if intent['tag'] not in classes:
                classes.append(intent['tag'])

    words = [nltk.WordNetLemmatizer().lemmatize(word.lower()) for word in words if word not in ignore_letters]
    words = sorted(set(words))
    classes = sorted(set(classes))

    return words, classes, documents

words, classes, documents = preprocess_intents(intents)

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [nltk.WordNetLemmatizer().lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25  # Seulement les prédictions avec une probabilité supérieure à ce seuil seront considérées
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            return result
    return "Désolé, je n'ai pas compris."

def speak(text):
    speaker.say(text)
    speaker.runAndWait()

def listen():
    with sr.Microphone() as mic:
        recognizer.adjust_for_ambient_noise(mic, duration=0.2)
        audio = recognizer.listen(mic)
        return recognizer.recognize_google(audio, language='fr')

def handle_command(message):
    ints = predict_class(message)
    if ints:
        intent = ints[0]['intent']

        # Ajouter une tâche
        if intent == 'add_todo':
            add_todo()
        # Afficher les tâches
        elif intent == 'show_todos':
            show_todos()
        else:
            response = get_response(ints, intents)
            update_textbox(f"Bot: {response}")
            speak(response)
    else:
        update_textbox("Bot: Je ne vous ai pas compris. Veuillez réessayer.")
        speak("Je ne vous ai pas compris. Veuillez réessayer.")

def update_textbox(message):
    textbox.insert(tk.END, message + "\n")
    textbox.yview(tk.END)

# Fonction pour créer une note
def create_note():
    global recognizer
    speaker.say("Que voulez-vous écrire dans votre note ?")
    speaker.runAndWait()

    done = False
    while not done:
        try:
            with sr.Microphone() as mic:
                recognizer.adjust_for_ambient_noise(mic, duration=0.2)
                audio = recognizer.listen(mic)
                note = recognizer.recognize_google(audio, language='fr')
                note = note.lower()
                speaker.say("Choisissez un nom pour le fichier !")
                speaker.runAndWait()
                recognizer.adjust_for_ambient_noise(mic, duration=0.2)
                audio = recognizer.listen(mic)
                filename = recognizer.recognize_google(audio, language='fr')
                filename = filename.lower()
            with open(filename + '.txt', 'w') as f:
                f.write(note)
                done = True
                speaker.say(f"Note créée avec succès : {filename}")
                speaker.runAndWait()
        except sr.UnknownValueError:
            recognizer = sr.Recognizer()
            speaker.say("Je ne vous ai pas compris! Veuillez réessayer!")
            speaker.runAndWait()

# Fonction pour ajouter une tâche à la liste
def add_todo():
    global recognizer
    speaker.say("Quelle tâche voulez-vous ajouter ?")
    speaker.runAndWait()

    done = False
    while not done:
        try:
            with sr.Microphone() as mic:
                recognizer.adjust_for_ambient_noise(mic, duration=0.2)
                audio = recognizer.listen(mic)
                item = recognizer.recognize_google(audio, language='fr')
                item = item.lower()
                todo_list.append(item)
                update_textbox(f"Bot: Tâche '{item}' ajoutée à la liste.")
                done = True
                speaker.say(f"{item} ajouté à la liste des tâches!")
                speaker.runAndWait()
        except sr.UnknownValueError:
            recognizer = sr.Recognizer()
            speaker.say("Je ne vous ai pas compris! Veuillez réessayer!")
            speaker.runAndWait()

# Fonction pour afficher les tâches
def show_todos():
    if todo_list:
        speaker.say("Les éléments dans la liste des tâches sont :")
        update_textbox("Bot: Les éléments dans la liste des tâches sont :")
        for item in todo_list:
            speaker.say(item)
            update_textbox(f" - {item}")
    else:
        speaker.say("La liste des tâches est vide.")
        update_textbox("Bot: La liste des tâches est vide.")
    speaker.runAndWait()

# Fonction pour quitter
def goodbye():
    speaker.say("Au revoir")
    speaker.runAndWait()
    sys.exit(0)

# Création de l'interface graphique
def on_speak_button_click():
    try:
        # Écoute et reconnaissance vocale
        message = listen().lower()
        update_textbox(f"User: {message}")
        # Traitement de la commande
        handle_command(message)
    except sr.UnknownValueError:
        speak("Je ne vous ai pas compris. Veuillez réessayer.")
        update_textbox("Bot: Je ne vous ai pas compris. Veuillez réessayer.")

# Fonction pour tester la synthèse vocale
def test_speech():
    speak("Bonjour")
    update_textbox("Bot: Bonjour")

# Initialiser l'application
app = tk.Tk()
app.title("Assistant Vocal")

# Zone de texte pour afficher les messages
textbox = scrolledtext.ScrolledText(app, wrap=tk.WORD, height=15, width=50)
textbox.pack(padx=10, pady=10)

# Bouton pour lancer la reconnaissance vocale
speak_button = tk.Button(app, text="Parler", command=on_speak_button_click)
speak_button.pack(padx=10, pady=5)

# Bouton pour tester la synthèse vocale
test_button = tk.Button(app, text="Tester la voix", command=test_speech)
test_button.pack(padx=10, pady=5)

# Bouton pour quitter l'application
exit_button = tk.Button(app, text="Quitter", command=app.quit)
exit_button.pack(padx=10, pady=5)

app.mainloop()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\oussa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\oussa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Shape of bags and output rows:
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
72 11
Epoch 1/200
9/9 [==============================] - 1s 4ms/step - loss: 2.4487 - accuracy: 0.0667
Epoch 2/200
9/9 [==============================] - 0s 2ms/step - loss: 2.3808 - accuracy: 0.0889
Epoch 3/200
9/9 [==============================] - 0s 3ms/step - loss: 2.3815 - accuracy: 0.1111
Epoch 4/200
9/9 [==============================] - 0s 2ms/step - loss: 2.3195 - accuracy: 0.1556
Epoch 5/200
9/9 [==============================] - 0s 3ms/step - loss: 2.2654 - accuracy: 0.2000
Epoch 6/200
9/9 [==============================] - 0s 3ms/step - loss: 2.2106 - accuracy: 0.3333
Epoch 7/200
9/9 [==============================] - 0s 2ms/step - loss: 2.1661 - accuracy: 0.3333
Epoch 8/200
9/9 [===

In [3]:
# تأكد من تنزيل الموارد اللازمة لـ NLTK
nltk.download('punkt')
nltk.download('wordnet')

# تهيئة الملمم
lemmatizer = nltk.WordNetLemmatizer()

# تحميل النوايا
with open('intents-arabe.json', 'r', encoding='utf-8') as file:
    intents = json.load(file)

words = []
classes = []
documents = []
ignore_letters = ['؟', '!', '.', ',']

# معالجة البيانات
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# تنقيح الكلمات وحذف علامات الترقيم
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

# حفظ الكلمات والفئات
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# تحضير بيانات التدريب
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

# تحقق من حجم المتجهات
print("شكل الحقيبة وصفوف المخرجات:")
for item in training:
    print(len(item[0]), len(item[1]))

# تحويل البيانات إلى مصفوفات NumPy
train_x = np.array([x for x, _ in training])
train_y = np.array([y for _, y in training])

# إنشاء النموذج
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# إنشاء خوارزمية التحسين SGD
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

# تجميع النموذج
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

# تدريب النموذج
hist = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

# حفظ النموذج
model.save('chatbot_model_ar.h5', hist)
print('تم الحفظ')

# تهيئة محرك تحويل النص إلى كلام
speaker = pyttsx3.init()

# تهيئة التعرف على الكلام
recognizer = sr.Recognizer()

# قائمة لتخزين المهام
todo_list = ["مهمة 1", "مهمة 2"]

# تحميل النوايا والنموذج
with open('intents-arabe.json', 'r', encoding='utf-8') as file:
    intents = json.load(file)
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model_ar.h5')

# معالجة البيانات
def preprocess_intents(intents):
    words = []
    classes = []
    documents = []
    ignore_letters = ['؟', '!', '.', ',']

    for intent in intents['intents']:
        for pattern in intent['patterns']:
            word_list = nltk.word_tokenize(pattern)
            words.extend(word_list)
            documents.append((word_list, intent['tag']))
            if intent['tag'] not in classes:
                classes.append(intent['tag'])

    words = [nltk.WordNetLemmatizer().lemmatize(word.lower()) for word in words if word not in ignore_letters]
    words = sorted(set(words))
    classes = sorted(set(classes))

    return words, classes, documents

words, classes, documents = preprocess_intents(intents)

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [nltk.WordNetLemmatizer().lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25  # سيتم فقط اعتبار التوقعات التي تتجاوز هذا العتبة
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            return result
    return "عذرًا، لم أفهم ذلك."

def speak(text):
    speaker.say(text)
    speaker.runAndWait()

def listen():
    with sr.Microphone() as mic:
        recognizer.adjust_for_ambient_noise(mic, duration=0.2)
        audio = recognizer.listen(mic)
        return recognizer.recognize_google(audio, language='ar')

def handle_command(message):
    ints = predict_class(message)
    if ints:
        intent = ints[0]['intent']

        # إضافة مهمة
        if intent == 'add_todo':
            add_todo()
        # عرض المهام
        elif intent == 'show_todos':
            show_todos()
        else:
            response = get_response(ints, intents)
            update_textbox(f"الروبوت: {response}")
            speak(response)
    else:
        update_textbox("الروبوت: لم أفهم ما قلته. حاول مرة أخرى.")
        speak("لم أفهم ما قلته. حاول مرة أخرى.")

def update_textbox(message):
    textbox.insert(tk.END, message + "\n")
    textbox.yview(tk.END)

# دالة لإنشاء ملاحظة
def create_note():
    global recognizer
    speaker.say("ماذا تريد أن تكتب في ملاحظتك؟")
    speaker.runAndWait()

    done = False
    while not done:
        try:
            with sr.Microphone() as mic:
                recognizer.adjust_for_ambient_noise(mic, duration=0.2)
                audio = recognizer.listen(mic)
                note = recognizer.recognize_google(audio, language='ar')
                note = note.lower()
                speaker.say("اختر اسمًا للملف!")
                speaker.runAndWait()
                recognizer.adjust_for_ambient_noise(mic, duration=0.2)
                audio = recognizer.listen(mic)
                filename = recognizer.recognize_google(audio, language='ar')
                filename = filename.lower()
            with open(filename + '.txt', 'w') as f:
                f.write(note)
                done = True
                speaker.say(f"تم إنشاء الملاحظة بنجاح: {filename}")
                speaker.runAndWait()
        except sr.UnknownValueError:
            recognizer = sr.Recognizer()
            speaker.say("لم أفهم! حاول مرة أخرى!")
            speaker.runAndWait()

# دالة لإضافة مهمة إلى القائمة
def add_todo():
    global recognizer
    speaker.say("ما المهمة التي تريد إضافتها؟")
    speaker.runAndWait()

    done = False
    while not done:
        try:
            with sr.Microphone() as mic:
                recognizer.adjust_for_ambient_noise(mic, duration=0.2)
                audio = recognizer.listen(mic)
                item = recognizer.recognize_google(audio, language='ar')
                item = item.lower()
                todo_list.append(item)
                update_textbox(f"الروبوت: تم إضافة '{item}' إلى القائمة.")
                done = True
                speaker.say(f"{item} تم إضافته إلى قائمة المهام!")
                speaker.runAndWait()
        except sr.UnknownValueError:
            recognizer = sr.Recognizer()
            speaker.say("لم أفهم! حاول مرة أخرى!")
            speaker.runAndWait()

# دالة لعرض المهام
def show_todos():
    if todo_list:
        speaker.say("العناصر في قائمة المهام هي:")
        update_textbox("الروبوت: العناصر في قائمة المهام هي:")
        for item in todo_list:
            speaker.say(item)
            update_textbox(f" - {item}")
    else:
        speaker.say("قائمة المهام فارغة.")
        update_textbox("الروبوت: قائمة المهام فارغة.")
    speaker.runAndWait()

# دالة للخروج
def goodbye():
    speaker.say("مع السلامة")
    speaker.runAndWait()
    sys.exit(0)

# إنشاء الواجهة الرسومية
def on_speak_button_click():
    try:
        # الاستماع والتعرف على الكلام
        message = listen().lower()
        update_textbox(f"المستخدم: {message}")
        # معالجة الأمر
        handle_command(message)
    except sr.UnknownValueError:
        speak("لم أفهم. حاول مرة أخرى.")
        update_textbox("الروبوت: لم أفهم. حاول مرة أخرى.")

# دالة لاختبار تحويل النص إلى كلام
def test_speech():
    speak("مرحبا")
    update_textbox("الروبوت: مرحبا")

# تهيئة التطبيق
app = tk.Tk()
app.title("المساعد الصوتي")

# مربع النص لعرض الرسائل
textbox = scrolledtext.ScrolledText(app, wrap=tk.WORD, height=15, width=50)
textbox.pack(padx=10, pady=10)

# زر لبدء التعرف على الكلام
speak_button = tk.Button(app, text="تحدث", command=on_speak_button_click)
speak_button.pack(padx=10, pady=5)

# زر لاختبار تحويل النص إلى كلام
test_button = tk.Button(app, text="اختبار الصوت", command=test_speech)
test_button.pack(padx=10, pady=5)

# زر للخروج من التطبيق
exit_button = tk.Button(app, text="خروج", command=app.quit)
exit_button.pack(padx=10, pady=5)

app.mainloop()


شكل الحقيبة وصفوف المخرجات:
35 6
35 6
35 6
35 6
35 6
35 6
35 6
35 6
35 6
35 6
35 6
35 6
35 6
35 6
35 6
35 6
35 6
35 6
35 6
35 6
35 6
Epoch 1/200


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\oussa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\oussa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


5/5 [==============================] - 1s 3ms/step - loss: 1.9513 - accuracy: 0.1429
Epoch 2/200
5/5 [==============================] - 0s 3ms/step - loss: 1.7290 - accuracy: 0.2857
Epoch 3/200
5/5 [==============================] - 0s 4ms/step - loss: 1.7496 - accuracy: 0.1905
Epoch 4/200
5/5 [==============================] - 0s 4ms/step - loss: 1.7462 - accuracy: 0.2381
Epoch 5/200
5/5 [==============================] - 0s 0s/step - loss: 1.7056 - accuracy: 0.3333
Epoch 6/200
5/5 [==============================] - 0s 4ms/step - loss: 1.6502 - accuracy: 0.3810
Epoch 7/200
5/5 [==============================] - 0s 4ms/step - loss: 1.6547 - accuracy: 0.4286
Epoch 8/200
5/5 [==============================] - 0s 6ms/step - loss: 1.5934 - accuracy: 0.4286
Epoch 9/200
5/5 [==============================] - 0s 3ms/step - loss: 1.4439 - accuracy: 0.5714
Epoch 10/200
5/5 [==============================] - 0s 5ms/step - loss: 1.5042 - accuracy: 0.5238
Epoch 11/200
5/5 [========================

In [5]:
import pyttsx3

speaker = pyttsx3.init()
speaker.say("مرحبا")
speaker.runAndWait()
